In [18]:
import unidecode
from nltk.translate.bleu_score import sentence_bleu
import random as rd

play_file_infixes = ['othello', 'antony-and-cleopatra', 'asyoulikeit', 
                     'errors', 'hamlet', 'henryv', 'juliuscaesar', 'lear', 'macbeth', 
                     'merchant', 'msnd', 'muchado', 'richardiii', 'romeojuliet', 
                     'shrew', 'tempest', 'twelfthnight']

agg_original_tuples = []
agg_modern_tuples = []
path = "data/shakespeare/data/align/plays/merged/"

for infix in play_file_infixes:
    modern_tuples = unidecode.unidecode(open(path + infix + "_modern.snt.aligned").read()).split("\n")
    original_tuples = unidecode.unidecode(open(path + infix + "_original.snt.aligned").read()).split("\n")
    agg_original_tuples.extend(original_tuples)
    agg_modern_tuples.extend(modern_tuples)
# aligned_othello_style1 = "data/shakespeare/data/align/plays/merged/othello_modern.snt.aligned"
# aligned_othello_style2 = "data/shakespeare/data/align/plays/merged/othello_original.snt.aligned"

# aligned_ant_cleo_style1 = "data/shakespeare/data/align/plays/merged/antony-and-cleopatra_modern.snt.aligned"
# aligned_ant_cleo_style2 = "data/shakespeare/data/align/plays/merged/antony-and-cleopatra_original.snt.aligned"

# aligned_asyoulikeit_style1 = "data/shakespeare/data/align/plays/merged/asyoulikeit_modern.snt.aligned"
# aligned_asyoulikeit_style2 = "data/shakespeare/data/align/plays/merged/asyoulikeit_original.snt.aligned"

# aligned_errors_style1 = "data/shakespeare/data/align/plays/merged/errors_modern.snt.aligned"
# aligned_errors_style2 = "data/shakespeare/data/align/plays/merged/errors_original.snt.aligned"

# aligned_hamlet_style1 = "data/shakespeare/data/align/plays/merged/hamlet_modern.snt.aligned"
# aligned_hamlet_style2 = "data/shakespeare/data/align/plays/merged/hamlet_original.snt.aligned"

# aligned_henryv_style1 = "data/shakespeare/data/align/plays/merged/henryv_modern.snt.aligned"
# aligned_henryv_style2 = "data/shakespeare/data/align/plays/merged/henryv_original.snt.aligned"

# aligned_juliuscaesar_style1 = "data/shakespeare/data/align/plays/merged/juliuscaesar_modern.snt.aligned"
# aligned_juliuscaesar_style2 = "data/shakespeare/data/align/plays/merged/juliuscaesar_original.snt.aligned"

# style1_tuples = unidecode.unidecode(open(aligned_style1).read())
# style2_tuples = unidecode.unidecode(open(aligned_style2).read())

# style1_tuples = style1_tuples.split("\n")
# style2_tuples = style2_tuples.split("\n")

In [33]:
print("{}\n{}\n".format(agg_original_tuples[0], agg_modern_tuples[0]))
print("{}\n{}\n".format(agg_original_tuples[5000], agg_modern_tuples[5000]))
print("{}\n{}\n".format(agg_original_tuples[9000], agg_modern_tuples[9000]))
print("{}\n{}\n".format(agg_original_tuples[18000], agg_modern_tuples[18000]))


What's the matter, lieutenant?
What's the matter, lieutenant?

Tell me this, I pray: Where have you left the money that I gave you?
Answer me this, please: where's the money I gave you?

Lucius, who's that knocking?
Lucius, who's that knocking?

Gentlemen, forward to the bridal dinner.
Gentlemen, on to the bridal dinner.



In [20]:
# in-class = original = '1'
# raw -> (raw text, label)
import numpy as np

style1_np = np.array(agg_original_tuples)
style1_labels = np.array(['1' for x in range(len(agg_original_tuples))])
X = np.vstack((style1_np, style1_labels))

style2_np = np.array(agg_modern_tuples)
style2_labels = np.array(['0' for x in range(len(agg_modern_tuples))])
X2 = np.vstack((style2_np, style2_labels))

X = np.hstack((X,X2))
raw = np.transpose(X)
raw.shape

idx_col = [str(x) for x in range(len(raw))]
idx_col = np.array(idx_col).reshape(-1,1)
print(idx_col.shape)
print(raw.shape)
raw = np.hstack((idx_col, raw))
raw.shape

(42192, 1)
(42192, 2)


(42192, 3)

In [21]:
raw[2000]

array(['2000', 'We looked not for Mark Antony here.', '1'], dtype='<U549')

In [22]:
# feature engineering
    # construct index column
idx_col = raw[:,0]
#idx_col.astype(float)

    
sent_len_feature = np.array([-1 for x in range(len(raw))])
raw_text_col = 1
LABEL_COL = -1

for row_idx,_ in enumerate(raw):
    sent_len_feature[row_idx] = len(raw[row_idx][raw_text_col])
#     if sent_len_feature[row_idx] < 0:
#         print("sent len less than zero!")

#sent_len_feature.astype(float)
X = np.vstack((idx_col,sent_len_feature, raw[:,LABEL_COL].astype(float)))
X = np.transpose(X)
X = X.astype('f', copy=True)
print(X[0:5])
print(X[3700:3705])

[[ 0. 30.  1.]
 [ 1. 42.  1.]
 [ 2.  8.  1.]
 [ 3. 23.  1.]
 [ 4. 21.  1.]]
[[3700.   27.    1.]
 [3701.   14.    1.]
 [3702.   17.    1.]
 [3703.   24.    1.]
 [3704.    7.    1.]]


In [23]:
X[0]

array([ 0., 30.,  1.], dtype=float32)

In [24]:
from sklearn.linear_model import LogisticRegression
import time
import math
# training
# test breakout: 3500, 210
train_test_split_delim = math.floor(len(X) * .8)
print("total rows: {} traintest split delim: {}".format(len(X), train_test_split_delim))
# numpy note: addressing [:delim] is exclusive of delim, but
#                         [delim:] is inclusive of delim

X = np.random.permutation(X)

X_train = X[0:train_test_split_delim,1]
X_train = X_train.reshape(-1, 1) #TODO: REMOVE
Y_train = X[0:train_test_split_delim,LABEL_COL]

X_test = X[train_test_split_delim:,1]
X_test = X_test.reshape(-1, 1)
Y_test = X[train_test_split_delim:,LABEL_COL]

logreg = LogisticRegression(random_state=123)
logreg.fit(X_train, Y_train)

np.set_printoptions(suppress=True)
X[0:10]

total rows: 42192 traintest split delim: 33753


array([[28189.,    25.,     0.],
       [29646.,    28.,     0.],
       [12591.,    38.,     1.],
       [19600.,    10.,     1.],
       [18172.,    41.,     1.],
       [  161.,    78.,     1.],
       [23185.,    55.,     0.],
       [28474.,   106.,     0.],
       [23380.,    26.,     0.],
       [27673.,    31.,     0.]], dtype=float32)

In [25]:
# evaluation
preds = logreg.predict(X_test)
test_indices = X[train_test_split_delim:,0]
if len(preds) != len(test_indices):
    raise ValueError("unmatched arrays")
    
results = []
for (idx, pred) in zip(test_indices,preds):
    gold_label = int(raw[int(idx)][-1])
    pred_label = int(pred)
    #print("int'd idx: {} gold label: {} pred label: {}".format(int(idx), gold_label, pred_label))
    
    if gold_label == pred_label:
          results.append(1)
    else:
        results.append(0)
    
results[:5]


[0, 0, 1, 1, 1]

In [26]:
test_len = len(preds)
accuracy = sum(preds)/float(test_len)
print("accuracy: {}".format(accuracy))

#othello accuracy: 0.7428571428571429. but maybe not permuted
#all: accuracy: 0.328237942884228


accuracy: 0.328237942884228


In [27]:
#x = np.zeros(3, dtype=[('row_id','int32'),('text', 'S'), ('label', 'int32')])

In [28]:
np.set_printoptions(suppress=True)

py_arr = [[  3701.,  30.,   1.]
 ,[  3702.,  53.,   1.]
 ,[  3703.,  15.,   1.]]


arr = np.array(py_arr)
arr = arr.astype(float)
np.random.shuffle(arr)
time.sleep(1)
arr[:3]

array([[3703.,   15.,    1.],
       [3701.,   30.,    1.],
       [3702.,   53.,    1.]])

In [29]:
arr.dtype

dtype('float64')

In [30]:
import sys
sys.executable

'/Users/peterkong/miniconda3/bin/python'

In [31]:
vr = "foo"
"bar" + vr

'barfoo'

42192